Results evaluation.

1) Linking google drive

2) Loading files (ground truth, results)

3) Combining ground truth and results for further analysis within Matlab


In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
#!git clone 'https://github.com/facebookresearch/detectron2'
#dist = distutils.core.run_setup("./detectron2/setup.py")
#!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
#sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-03ub67r4
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-03ub67r4
  Resolved https://github.com/facebookresearch/detectron2.git to commit 38af375052d3ae7331141bc1a22cfa2713b02987
  Preparing metadata (setup.py) ... done


In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  1.13 ; cuda:  cu116


In [4]:
#Initialization
#For Detectron2
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

#Import useful Detectron2 functions
from detectron2.structures import pairwise_iou, Boxes, BoxMode

## Linking to google drive. Note that we require google-drive-ocamlfuse and authentication when using GCP.

In [5]:
##Only need to do this once for installation.
#!sudo apt-get install google-drive-ocamlfuse
#!sudo add-apt-repository ppa:alessandro-strada/ppa
#!sudo apt-get update
#!sudo apt-get install google-drive-ocamlfuse

In [6]:
##Not used during GCP mode 
from google.colab import drive
drive.mount('/content/gdrive/')
os.chdir('./gdrive/MyDrive/Colab Notebooks/AppliedCV/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [7]:
#Authentication (follow instructions and copy code to access google drive)
#!google-drive-ocamlfuse -headless -id=553698332080-4ilebkr6tajj9d4on76a6sfjilo8dbob.apps.googleusercontent.com -secret=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

In [8]:
#Initializing google drive to a folder
#if not os.path.exists("./acv"):
#  os.makedirs("./acv")

#!google-drive-ocamlfuse acv

In [9]:
#Change director to personal folder within the drive
#os.chdir('./acv/Colab Notebooks/AppliedCV/')

## Loading files

In [10]:
#Initialization
img_inshore = './Dataset/SSDD/Images/test_inshore.txt'
img_offshore = './Dataset/SSDD/Images/test_offshore.txt'
gt = './Dataset/SSDD/labels.csv'
res_frcnnR = './ModelOutput/faster_rcnn_R_101_FPN_3x_3_r1/coco_instances_results.json'

In [11]:
#Loading results and ground truth 
df_inshore = pd.read_csv(img_inshore, header=None, names=['filename'])
df_offshore = pd.read_csv(img_offshore, header=None, names=['filename'])
df_gt = pd.read_csv(gt)
df_frcnnR = pd.read_json(res_frcnnR,dtype={'image_id': 'str'})

In [12]:
#Cleaning for ground truth
ind = np.where(df_gt['filename'].astype(str).str[0:2].astype(int) <= 10)
df_gt = df_gt.drop(ind[0])
df_gt['filename'] = df_gt['filename'].str[:-4]
df_gt['inshore'] = 0
df_gt['result'] = 0
df_gt['det_id'] = 0
print(df_gt)
print(len(df_gt))

df_frcnnR['inshore'] = 0

      filename  width  height class  xmin  ymin  xmax  ymax  inshore  result  \
0     14_19_11    800     800  ship    14   535    35   557        0       0   
1     14_19_11    800     800  ship   406   389   420   402        0       0   
2     14_19_11    800     800  ship   392   689   402   699        0       0   
3     14_15_23    800     800  ship   683   275   717   313        0       0   
4     14_16_13    800     800  ship   177   368   199   407        0       0   
...        ...    ...     ...   ...   ...   ...   ...   ...      ...     ...   
3040  11_14_28    800     800  ship    58   365    78   394        0       0   
3041  11_14_28    800     800  ship   294   363   315   387        0       0   
3042  11_14_28    800     800  ship   230   274   256   291        0       0   
3043  11_14_28    800     800  ship   339   265   362   303        0       0   
3044  11_14_28    800     800  ship   434   106   453   126        0       0   

      det_id  
0          0  
1        

In [13]:
#Adding 'inshore' header
for i in range(len(df_inshore)):  
  txt = df_inshore['filename'].iloc[i]
  ind = np.where(df_gt['filename'] == txt)
  if len(ind) > 0:
    df_gt['inshore'].iloc[ind] = 1
  
  ind = np.where(df_frcnnR['image_id'] == txt)
  if len(ind) > 0:
    df_frcnnR['inshore'].iloc[ind] = 1

print(df_gt)
print(df_frcnnR)

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


      filename  width  height class  xmin  ymin  xmax  ymax  inshore  result  \
0     14_19_11    800     800  ship    14   535    35   557        0       0   
1     14_19_11    800     800  ship   406   389   420   402        0       0   
2     14_19_11    800     800  ship   392   689   402   699        0       0   
3     14_15_23    800     800  ship   683   275   717   313        0       0   
4     14_16_13    800     800  ship   177   368   199   407        0       0   
...        ...    ...     ...   ...   ...   ...   ...   ...      ...     ...   
3040  11_14_28    800     800  ship    58   365    78   394        0       0   
3041  11_14_28    800     800  ship   294   363   315   387        0       0   
3042  11_14_28    800     800  ship   230   274   256   291        0       0   
3043  11_14_28    800     800  ship   339   265   362   303        0       0   
3044  11_14_28    800     800  ship   434   106   453   126        0       0   

      det_id  
0          0  
1        

In [14]:
df_frcnnR

,image_id,category_id,bbox,score,inshore
0,11_10_10,0,"[483.7619323730469, 397.259765625, 16.08984375...",0.884446,0
1,11_10_11,0,"[392.0898132324219, 411.69512939453125, 15.798...",0.924196,0
2,11_10_13,0,"[362.3845520019531, 28.692062377929688, 25.829...",0.958484,0
3,11_10_13,0,"[161.8732452392578, 378.771484375, 15.75973510...",0.927231,0
4,11_10_15,0,"[351.986328125, 304.61260986328125, 23.2286987...",0.936764,0
...,...,...,...,...,...
2170,15_7_24,0,"[374.9517822265625, 307.6913757324219, 7.28149...",0.647554,1
2171,15_8_9,0,"[414.095458984375, 502.5772399902344, 18.30242...",0.931564,0
2172,15_9_12,0,"[380.8478088378906, 252.2923126220703, 20.2971...",0.924325,0
2173,15_9_15,0,"[660.5133666992188, 30.971466064453125, 35.711...",0.976731,0


In [15]:
#Adding more fields to ground truth
df_gt['bbox'] = df_gt[['xmin', 'ymin','xmax','ymax']].values.tolist()

df_gt['iou'] = 0 
print(df_gt)

      filename  width  height class  xmin  ymin  xmax  ymax  inshore  result  \
0     14_19_11    800     800  ship    14   535    35   557        0       0   
1     14_19_11    800     800  ship   406   389   420   402        0       0   
2     14_19_11    800     800  ship   392   689   402   699        0       0   
3     14_15_23    800     800  ship   683   275   717   313        0       0   
4     14_16_13    800     800  ship   177   368   199   407        0       0   
...        ...    ...     ...   ...   ...   ...   ...   ...      ...     ...   
3040  11_14_28    800     800  ship    58   365    78   394        0       0   
3041  11_14_28    800     800  ship   294   363   315   387        0       0   
3042  11_14_28    800     800  ship   230   274   256   291        0       0   
3043  11_14_28    800     800  ship   339   265   362   303        0       0   
3044  11_14_28    800     800  ship   434   106   453   126        0       0   

      det_id                  bbox  iou

In [16]:
#Sanity check
print(len(df_gt[df_gt['inshore'] == 1]))
print(len(df_gt[df_gt['inshore'] == 0]))

883
1495


In [17]:
#Initialization of results
df_frcnnR['result'] = 0 #0: FA, 1: Detected, 2: Double counted

df_frcnnR.bbox.map(lambda x: x[0]) + df_frcnnR.bbox.map(lambda x: x[2])

In [19]:
#Adding additional fields for results
df_frcnnR['xmin'] = df_frcnnR.bbox.map(lambda x: x[0])
df_frcnnR['xmax'] = df_frcnnR.bbox.map(lambda x: x[0]) + df_frcnnR.bbox.map(lambda x: x[2])
df_frcnnR['ymin'] = df_frcnnR.bbox.map(lambda x: x[1])
df_frcnnR['ymax'] = df_frcnnR.bbox.map(lambda x: x[1]) + df_frcnnR.bbox.map(lambda x: x[3])

df_frcnnR['bbox2'] = df_frcnnR[['xmin', 'ymin','xmax','ymax']].values.tolist()

df_frcnnR['iou'] = 0
df_frcnnR['gt_id'] = 0
print(df_frcnnR)

      image_id  category_id  \
0     11_10_10            0   
1     11_10_11            0   
2     11_10_13            0   
3     11_10_13            0   
4     11_10_15            0   
...        ...          ...   
2170   15_7_24            0   
2171    15_8_9            0   
2172   15_9_12            0   
2173   15_9_15            0   
2174    15_9_9            0   

                                                   bbox     score  inshore  \
0     [483.7619323730469, 397.259765625, 16.08984375...  0.884446        0   
1     [392.0898132324219, 411.69512939453125, 15.798...  0.924196        0   
2     [362.3845520019531, 28.692062377929688, 25.829...  0.958484        0   
3     [161.8732452392578, 378.771484375, 15.75973510...  0.927231        0   
4     [351.986328125, 304.61260986328125, 23.2286987...  0.936764        0   
...                                                 ...       ...      ...   
2170  [374.9517822265625, 307.6913757324219, 7.28149...  0.647554        1   
217

In [20]:
df_frcnnR

,image_id,category_id,bbox,score,inshore,result,xmin,xmax,ymin,ymax,bbox2,iou,gt_id
0,11_10_10,0,"[483.7619323730469, 397.259765625, 16.08984375...",0.884446,0,0,483.761932,499.851776,397.259766,413.278076,"[483.7619323730469, 397.259765625, 499.8517761...",0,0
1,11_10_11,0,"[392.0898132324219, 411.69512939453125, 15.798...",0.924196,0,0,392.089813,407.888153,411.695129,430.935059,"[392.0898132324219, 411.69512939453125, 407.88...",0,0
2,11_10_13,0,"[362.3845520019531, 28.692062377929688, 25.829...",0.958484,0,0,362.384552,388.214447,28.692062,49.478523,"[362.3845520019531, 28.692062377929688, 388.21...",0,0
3,11_10_13,0,"[161.8732452392578, 378.771484375, 15.75973510...",0.927231,0,0,161.873245,177.632980,378.771484,397.900024,"[161.8732452392578, 378.771484375, 177.6329803...",0,0
4,11_10_15,0,"[351.986328125, 304.61260986328125, 23.2286987...",0.936764,0,0,351.986328,375.215027,304.612610,340.823486,"[351.986328125, 304.61260986328125, 375.215026...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,15_7_24,0,"[374.9517822265625, 307.6913757324219, 7.28149...",0.647554,1,0,374.951782,382.233276,307.691376,316.428986,"[374.9517822265625, 307.6913757324219, 382.233...",0,0
2171,15_8_9,0,"[414.095458984375, 502.5772399902344, 18.30242...",0.931564,0,0,414.095459,432.397888,502.577240,520.693176,"[414.095458984375, 502.5772399902344, 432.3978...",0,0
2172,15_9_12,0,"[380.8478088378906, 252.2923126220703, 20.2971...",0.924325,0,0,380.847809,401.144989,252.292313,267.453156,"[380.8478088378906, 252.2923126220703, 401.144...",0,0
2173,15_9_15,0,"[660.5133666992188, 30.971466064453125, 35.711...",0.976731,0,0,660.513367,696.224548,30.971466,57.783913,"[660.5133666992188, 30.971466064453125, 696.22...",0,0


## Quantitative results generation

In [22]:
#Computing scores and linking gt and results for offshore case.
for i in range(len(df_offshore)):
  bbox_gt = []
  bbox_frcnnR = []
  #print(i)
  txt = df_offshore['filename'].iloc[i]
  ind = np.where(df_gt['filename'] == txt) #Find ground truth in df_gt
  #print(txt)
  #print(len(ind[0]))
  #print(ind[0])
  if len(ind[0]) > 0:
    bbox_gt = df_gt['bbox'].iloc[ind[0]]
    #print(bbox_gt)
  ind2 = np.where(df_frcnnR['image_id'] == txt) #Find detections in df_frcnnR
  if len(ind2[0]) > 0:
    bbox_frcnnR = df_frcnnR['bbox2'].iloc[ind2[0]]
    #print(bbox_frcnnR)
  
  #for j in range(ind): #For each detection
  #print(Boxes(bbox_gt.tolist()))
  #print(Boxes(bbox_frcnnR.tolist()))
  if len(bbox_gt) == 0 or len(bbox_frcnnR) == 0:
    continue
  overlap = pairwise_iou(Boxes(bbox_gt.tolist()), Boxes(bbox_frcnnR.tolist()))
  for j in range(overlap.shape[0]):
    #print(j)
    arr = overlap[j,:]
    #print(arr)
    maxI = np.argmax(arr)
    if arr[maxI] == 0:
      continue
    allI = [n for n,x in enumerate(arr) if x>0.001]
    if len(allI) == 0:
      continue
    #print(ind)
    #print(arr[ind])  
    df_gt['result'].iloc[ind[0][j]] = 1
    df_gt['det_id'].iloc[ind[0][j]] = ind2[0][maxI]
    df_gt['iou'].iloc[ind[0][j]] = float(arr[maxI]) #Score IOU
    df_frcnnR['result'].iloc[ind2[0][maxI]] = 1
    df_frcnnR['iou'].iloc[ind2[0][maxI]] = float(arr[maxI]) #Score IOU
    df_frcnnR['gt_id'].iloc[ind2[0][maxI]] = ind[0][j]
    for k in allI:
      if k != maxI:
        df_frcnnR['result'].iloc[ind2[0][k]] = 2 #Overlapping


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [24]:
#Computing scores and linking gt and results for inshore case.
for i in range(len(df_inshore)):
  bbox_gt = []
  bbox_frcnnR = []
  #print(i)
  txt = df_inshore['filename'].iloc[i]
  ind = np.where(df_gt['filename'] == txt) #Find ground truth in df_gt
  #print(txt)
  #print(len(ind[0]))
  #print(ind[0])
  if len(ind[0]) > 0:
    bbox_gt = df_gt['bbox'].iloc[ind[0]]
    #print(bbox_gt)
  ind2 = np.where(df_frcnnR['image_id'] == txt) #Find detections in df_frcnnR
  if len(ind2[0]) > 0:
    bbox_frcnnR = df_frcnnR['bbox2'].iloc[ind2[0]]
    #print(bbox_frcnnR)
  
  #for j in range(ind): #For each detection
  #print(Boxes(bbox_gt.tolist()))
  #print(Boxes(bbox_frcnnR.tolist()))
  if len(bbox_gt) == 0 or len(bbox_frcnnR) == 0:
    continue
  overlap = pairwise_iou(Boxes(bbox_gt.tolist()), Boxes(bbox_frcnnR.tolist()))
  for j in range(overlap.shape[0]):
    #print(j)
    arr = overlap[j,:]
    #print(arr)
    maxI = np.argmax(arr)
    if arr[maxI] == 0:
      continue
    allI = [n for n,x in enumerate(arr) if x>0.001]
    if len(allI) == 0:
      continue
    #print(ind)
    #print(arr[ind])  
    df_gt['result'].iloc[ind[0][j]] = 1
    df_gt['det_id'].iloc[ind[0][j]] = ind2[0][maxI]
    df_gt['iou'].iloc[ind[0][j]] = float(arr[maxI]) #Score IOU
    df_frcnnR['result'].iloc[ind2[0][maxI]] = 1
    df_frcnnR['iou'].iloc[ind2[0][maxI]] = float(arr[maxI]) #Score IOU
    df_frcnnR['gt_id'].iloc[ind2[0][maxI]] = ind[0][j]
    for k in allI:
      if k != maxI:
        df_frcnnR['result'].iloc[ind2[0][k]] = 2 #Overlapping


In [26]:
#Saving results for further processing in Matlab
df_gt.to_csv('./gt_res.csv')
df_frcnnR.to_csv('./frcnnR_res.csv')